In [ ]:
import os
import sys
import seaborn as sns

# import pyLDAvis
import pyLDAvis.gensim_models

import numpy as np

import pandas as pd
from gensim import corpora, models
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
# Add parent directory to path to import modules from src
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
from src.preprocessing import Preprocessing
# Initialize DataLoader
sl = SlackDataLoader('../Anonymized_B6SlackExport_25Nov23/anonymized/')
# converting all json files for all-community-building channel to dataframe using slack_parser method from loader script
df_community = sl.slack_parser('../Anonymized_B6SlackExport_25Nov23/anonymized/all-community-building/')

In [ ]:
# Convert 'message_sent_time' to datetime format
# df['message_sent_time'] = pd.to_datetime(df['message_sent_time'])
df_community['msg_sent_time'] = pd.to_datetime(df_community['msg_sent_time'], unit='s')

# Sort the DataFrame by 'message_sent_time'
df_community = df_community.sort_values(by='msg_sent_time')

# Calculate time differences between consecutive messages
df_community['time_difference'] = df_community['msg_sent_time'].diff()

# Plot the distribution of time differences
plt.hist(df_community['time_difference'].dt.total_seconds(), bins=50, edgecolor='black')
plt.xlabel('Time Differences (seconds)')
plt.ylabel('Frequency')
plt.title('Distribution of Time Differences Between Messages')
plt.show()

In [ ]:
# Find the index of the maximum reply_count
max_reply_count= df_community['reply_count'].max()
max_reply_count

In [ ]:
msg_content = sl.get_channel_messages(channel_name='all-community-building')
def filter_msg(msg_content):
    
    for msg in msg_content:
        if 'reply_count' in msg.keys():
            if msg['reply_count']==75:
                return msg
top_reply_count_message = filter_msg(msg_content)
# convery replies key to a dataframe
df_replies = pd.DataFrame(top_reply_count_message['replies'])
df_replies.columns

In [ ]:
df_replies['ts'] = pd.to_datetime(df_replies['ts'], unit='s')

# Sort the DataFrame by 'message_sent_time'
df_replies = df_replies.sort_values(by='ts')

# Calculate time differences between consecutive messages
df_replies['time_difference'] = df_replies['ts'].diff()

# Plot the distribution of time differences
plt.hist(df_replies['time_difference'].dt.total_seconds(), bins=50, edgecolor='black')
plt.xlabel('Time Differences (seconds)')
plt.ylabel('Frequency')
plt.title('Distribution of Time Differences Between Messages')
plt.show()

In [ ]:
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity
from gensim.utils import SaveLoad
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from re import sub
import pyLDAvis
from collections import Counter
from gensim.matutils import corpus2csc, sparse2full, corpus2dense
from wordcloud import WordCloud
from sklearn.utils import resample
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
df_community.head()

In [ ]:
preprocess = Preprocessing(df_community)
from nltk.corpus import stopwords
stop = stopwords.words('english')

# clean text 
df_community = preprocess.cleantext('msg_content', 'clean_msg_content')

# Stem words
df_community = preprocess.stem('clean_msg_content')

In [ ]:
# Used stemmer instead of lemmatizer
cleanKagslacklist = preprocess.filterSlackList(df_community['clean_msg_content'])
cleanKagslacklist[1]
